In [ ]:
pip install --upgrade democlassi

     |████████████████████████████████| 10.1 MB 4.3 MB/s 
     |████████████████████████████████| 22.3 MB 67.3 MB/s 
     |████████████████████████████████| 17.4 MB 22.7 MB/s 
     |████████████████████████████████| 103 kB 57.7 MB/s 
     |████████████████████████████████| 804.1 MB 2.4 kB/s 
     |████████████████████████████████| 125 kB 61.1 MB/s 
     |████████████████████████████████| 3.0 MB 36.2 MB/s 
     |████████████████████████████████| 29.2 MB 1.3 MB/s 
     |████████████████████████████████| 7.4 MB 23.6 MB/s 


In [ ]:
import torch
import torchvision.transforms as transforms

from vision_utils.custom_torch_utils import load_model
from vision_utils.custom_architectures import SepConvModelMT, SepConvModel, initialize_model,PretrainedMT

from emotion_detection.evaluate import evaluate_model as eval_fer
from emotion_detection.fer_data_utils import get_fer_dataloader
from emotion_detection.train import run_fer

from multitask_rag.train import run_utk
from multitask_rag.utk_data_utils import get_utk_dataloader
from multitask_rag.evaluate import evaluate_model as eval_utk
from multitask_rag.utk_data_utils import display_examples_utk

import pandas as pd
import glob
import os
import random

In [ ]:
path_fer_resnet = r'/Users/ikea/Documents/DKU/STATS402/resnet_model_21_val_loss=4.275671.pth'

In [ ]:
model = PretrainedMT(model_name='resnet').to(device)

In [ ]:
model.module.load_state_dict(torch.load(path_fer_resnet, map_location='cpu'))

<All keys matched successfully>

In [ ]:
model

DataParallel(
  (module): PretrainedMT(
    (conv_base): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): Re

In [ ]:
path = "/Users/ikea/Documents/DKU/STATS402/hateful_memes/"

In [ ]:
import pandas as pd
dev_seen= pd.read_json(path+'dev_seen.jsonl',lines=True)
dev_unseen= pd.read_json(path+'dev_unseen.jsonl',lines=True)
test_seen=pd.read_json(path+'test_seen.jsonl',lines=True)
test_unseen=pd.read_json(path+'test_unseen.jsonl',lines=True)
train=pd.read_json(path+'train.jsonl',lines=True)

In [ ]:
facedata = pd.read_csv('/Users/ikea/Documents/DKU/STATS402/facetype.csv')
facedata

,Img,Facetype
0,01235.png,NaN
1,01236.png,NaN
2,01243.png,0.0
3,01245.png,NaN
4,01247.png,NaN
...,...,...
12135,98752.png,0.0
12136,98754.png,NaN
12137,98756.png,NaN
12138,98762.png,NaN


In [ ]:
data = pd.concat([train,test_seen,test_unseen,dev_seen,dev_unseen])
data

,id,img,label,text
0,42953,img/42953.png,0,its their character not their color that matters
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...
2,13894,img/13894.png,0,putting bows on your pet
3,37408,img/37408.png,0,i love everything and everybody! except for sq...
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h..."
...,...,...,...,...
535,83675,img/83675.png,0,i'm gonna be like phelps one day
536,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...
537,48670,img/48670.png,0,look at this sandwich maker club i found on wi...
538,9863,img/09863.png,0,diverse group of women


In [ ]:
data1 = data.drop_duplicates(subset=['img'])
np.dtype(data1['id'])

dtype('int64')

In [ ]:
facedata['id']=[int(x[:-4]) for x in facedata['Img']]
facetype = facedata[facedata['Facetype']==0]
facedata

,Img,Facetype,id
0,01235.png,NaN,1235
1,01236.png,NaN,1236
2,01243.png,0.0,1243
3,01245.png,NaN,1245
4,01247.png,NaN,1247
...,...,...,...
12135,98752.png,0.0,98752
12136,98754.png,NaN,98754
12137,98756.png,NaN,98756
12138,98762.png,NaN,98762


In [ ]:
facetype = pd.merge(data1, facedata, on=['id'])
facetype

,id,img,label,text,Img,Facetype
0,42953,img/42953.png,0,its their character not their color that matters,42953.png,NaN
1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,23058.png,NaN
2,13894,img/13894.png,0,putting bows on your pet,13894.png,0.0
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,37408.png,0.0
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403.png,0.0
...,...,...,...,...,...,...
12135,82054,img/82054.png,0,puzzle level expert,82054.png,0.0
12136,13749,img/13749.png,0,it's amazing how animal bodies and the human b...,13749.png,NaN
12137,85067,img/85067.png,0,rare image of a worker picking cotton,85067.png,0.0
12138,20541,img/20541.png,0,caught me cat and monkey today! this,20541.png,0.0


In [ ]:
facetype0 = facetype[facetype['Facetype']==0]
facetype0

,id,img,label,text,Img,Facetype
2,13894,img/13894.png,0,putting bows on your pet,13894.png,0.0
3,37408,img/37408.png,0,i love everything and everybody! except for sq...,37408.png,0.0
4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...",82403.png,0.0
5,16952,img/16952.png,0,go sports! do the thing! win the points!,16952.png,0.0
6,76932,img/76932.png,0,fine you're right. now can we fucking drop it?,76932.png,0.0
...,...,...,...,...,...,...
12130,31702,img/31702.png,0,the nazis killed millions of jewish people,31702.png,0.0
12133,27054,img/27054.png,0,now that's a little savage little savage,27054.png,0.0
12135,82054,img/82054.png,0,puzzle level expert,82054.png,0.0
12137,85067,img/85067.png,0,rare image of a worker picking cotton,85067.png,0.0


In [ ]:
data_0 = facetype0[facetype0['label']==0]
data_1 = facetype0[facetype0['label']==1]

In [ ]:
import shutil
for index, row in data_0.iterrows():
    path = '/Users/ikea/Documents/DKU/STATS402/hateful_memes/'+row['img']
    target = '/Users/ikea/Documents/DKU/STATS402/face/image0/' +str(row['id'])+'.png'
    shutil.copy(path,target) 

In [ ]:
for index, row in data_1.iterrows():
    path = '/Users/ikea/Documents/DKU/STATS402/hateful_memes/'+row['img']
    target = '/Users/ikea/Documents/DKU/STATS402/face/image1/' +str(row['id'])+'.png'
    shutil.copy(path,target) 

In [ ]:
import torch
import torch.nn.functional as F
import tqdm
import numpy as np
from vision_utils.custom_torch_utils import plot_confusion_matrix
from vision_utils.custom_torch_utils import processing_time
from torchvision import transforms
import time

In [ ]:
data_path_test = "/Users/ikea/Documents/DKU/STATS402/face"
test_dataset = torchvision.datasets.ImageFolder(root=data_path_test,
                                                transform=transforms.Compose([transforms.Resize((128, 128)),
                                                transforms.ToTensor()]))
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=24,
                                          shuffle=False)

In [ ]:
def evaluate_model(model, dataloader,
                   labels_=[[0, 1], [0, 1, 2, 3, 4]],
                   target_names=[['Male', 'Female'], ['White', 'Black', 'Asian', 'Indian', 'Unknown']],
                   normalize=False):
    """
        Function for evaluating a classification model by printing/plotting classification report and confusion matrix

        :param model: a pytorch trained model
        :param dataloader: a pytorch DataLoader object, or any object that yields pytorch tensors
                ready to be used by the model
        :param title: a string to be used as the plot title
        :param labels_: list  of lists , each sublist is a list of integers (0 to number of classes - 1) representing
                        labels for an output from the model
        :param target_names: list of lists, each sublist is a list of strings or ints that describe the labels,
                            and must have the same length as the corresponding labels it describes from `labels`list
        :param normalize: whether to show the actual values or in % for the confusion matrix
        :return:
        """

    y_age = []
    y_gender = []
    y_race = []
    y_pred_age = []
    y_pred_gender = []
    y_pred_race = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # first, get the predictions
    model.eval()  # set model in evaluation mode
    model = model.to(device)

    with torch.no_grad():
        # Iterate over data.
        for images, labels in tqdm.tqdm(dataloader):
            images = images.to(device, dtype=torch.float32)

            age_pred, gender_pred, race_pred = model(images)
            y_pred_age.append(age_pred.to('cpu').numpy())
            _, gender_pred = torch.max(gender_pred, 1)
            _, race_pred = torch.max(race_pred, 1)
            y_pred_gender.append(gender_pred.to('cpu').numpy())
            y_pred_race.append(race_pred.to('cpu').numpy())
    return y_pred_age,y_pred_gender,y_pred_race

In [ ]:
y_pred_age,y_pred_gender,y_pred_race = evaluate_model(model, test_loader)

100%|█████████████████████████████████████████| 160/160 [05:28<00:00,  2.05s/it]


In [ ]:
imgs_pred = dict()
for i in range(len(label)):
    imgs_pred[test_dataset.imgs[i][0]] = predict[i].tolist()

In [ ]:
y_pred_race

[array([0, 0, 1, 2, 1, 0, 0, 2, 1, 0, 0, 2, 0, 2, 2, 0, 0, 0, 1, 0, 0, 0,
        2, 0]),
 array([2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 0, 2, 0, 0,
        0, 1]),
 array([0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 0, 0, 2, 1, 0, 1, 2, 0,
        0, 0]),
 array([1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2,
        2, 1]),
 array([2, 0, 0, 1, 3, 2, 0, 0, 0, 2, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0,
        2, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 2, 0, 2, 0, 0,
        0, 0]),
 array([0, 0, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1,
        1, 1]),
 array([0, 1, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]),
 array([1, 0, 2, 0, 0, 2, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 1, 1, 3, 0, 0, 0,
        2, 2]),
 array([0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2,
        1, 1]),
 array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 2,
        0, 1]),
 array([0,